## Initialize

In [1]:
import os
import shutil

import numpy as np
import pandas as pd

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

### Target and LFP directory

In [3]:
data_directory ="example_ecephys_project_cache" # must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

LFP_directory = "E:\Elham\Data\AllenBrain\example_ecephys_project_cache\LFP"


In [12]:
os.path.exists(LFP_directory)

True

### Read session IDs and their probe IDs

In [14]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)
sessions = cache.get_session_table()

In [15]:
Filtered_sessions = sessions[(sessions.full_genotype=='wt/wt') & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISp' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

In [16]:
probes = cache.get_probes()

### Download the files

In [20]:
from allensdk.brain_observatory.ecephys.ecephys_project_api.utilities import build_and_execute
from allensdk.brain_observatory.ecephys.ecephys_project_api.rma_engine import RmaEngine
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

rma_engine = RmaEngine(scheme="http", host="api.brain-map.org")

First download the sessions

In [21]:
def retrieve_link(session_id):
    
    well_known_files = build_and_execute(
        (
        "criteria=model::WellKnownFile"
        ",rma::criteria,well_known_file_type[name$eq'EcephysNwb']"
        "[attachable_type$eq'EcephysSession']"
        r"[attachable_id$eq{{session_id}}]"
        ),
        engine=rma_engine.get_rma_tabular, 
        session_id=session_id
    )
    
    return 'http://api.brain-map.org/' + well_known_files['download_link'].iloc[0]

download_links = [retrieve_link(session_id) for session_id in Filtered_sessions.index.values]

In [26]:
_ = [print(link) for link in download_links]

http://api.brain-map.org//api/v2/well_known_file_download/1026123599
http://api.brain-map.org//api/v2/well_known_file_download/1026123989
http://api.brain-map.org//api/v2/well_known_file_download/1026123897
http://api.brain-map.org//api/v2/well_known_file_download/1026123964
http://api.brain-map.org//api/v2/well_known_file_download/1026124068
http://api.brain-map.org//api/v2/well_known_file_download/1026124429
http://api.brain-map.org//api/v2/well_known_file_download/1026124724
http://api.brain-map.org//api/v2/well_known_file_download/1026124242
http://api.brain-map.org//api/v2/well_known_file_download/1026123537
http://api.brain-map.org//api/v2/well_known_file_download/1026124326
http://api.brain-map.org//api/v2/well_known_file_download/1026124603
http://api.brain-map.org//api/v2/well_known_file_download/1026123685
http://api.brain-map.org//api/v2/well_known_file_download/1026123377
http://api.brain-map.org//api/v2/well_known_file_download/1026124793
http://api.brain-map.org//api/v2/w

In [ ]:
import requests
r = requests.get(download_links[1])

You should first download these and organize the folders. Can be added later.

In [28]:
Filtered_probes = probes[[ID in Filtered_sessions.index.values for ID in probes.ecephys_session_id]]
def retrieve_lfp_link(probe_id):

    well_known_files = build_and_execute(
        (
            "criteria=model::WellKnownFile"
            ",rma::criteria,well_known_file_type[name$eq'EcephysLfpNwb']"
            "[attachable_type$eq'EcephysProbe']"
            r"[attachable_id$eq{{probe_id}}]"
        ),
        engine=rma_engine.get_rma_tabular, 
        probe_id=probe_id
    )

    if well_known_files.shape[0] != 1:
        return 'file for probe ' + str(probe_id) + ' not found'
        
    return 'http://api.brain-map.org/' + well_known_files.loc[0, "download_link"]

probes = cache.get_probes()

download_links = [retrieve_lfp_link(probe_id) for probe_id in Filtered_probes.index.values]

_ = [print(link) for link in download_links]

http://api.brain-map.org//api/v2/well_known_file_download/1026123601
http://api.brain-map.org//api/v2/well_known_file_download/1026123603
http://api.brain-map.org//api/v2/well_known_file_download/1026123605
http://api.brain-map.org//api/v2/well_known_file_download/1026123607
http://api.brain-map.org//api/v2/well_known_file_download/1026123609
http://api.brain-map.org//api/v2/well_known_file_download/1026123539
http://api.brain-map.org//api/v2/well_known_file_download/1026123543
http://api.brain-map.org//api/v2/well_known_file_download/1026123550
http://api.brain-map.org//api/v2/well_known_file_download/1026123546
http://api.brain-map.org//api/v2/well_known_file_download/1026123541
http://api.brain-map.org//api/v2/well_known_file_download/1026123548
http://api.brain-map.org//api/v2/well_known_file_download/1026124433
http://api.brain-map.org//api/v2/well_known_file_download/1026124436
http://api.brain-map.org//api/v2/well_known_file_download/1026124439
http://api.brain-map.org//api/v2/w

#### Read  the LFP names and find their target animals

In [119]:
LFP_directory = "E:\Elham\Data\AllenBrain\example_ecephys_project_cache\LFP"

In [120]:
LFP_list = os.listdir(LFP_directory)

In [121]:
probe_ids = [int(x.split('_')[1]) for x in LFP_list]

In [122]:
probe_session = probes.loc[np.array(probe_ids),'ecephys_session_id'].to_frame().reset_index()

copy the LFP files in their target directory

In [ ]:
for i in range(0,len(probe_session)):
    #print(i)
    copyLFP(probe_session.ecephys_session_id[i],probe_session.id[i],LFP_directory,data_directory,False)

In [116]:
def copyLFP(session_id,probe_id,sourcepath,targetpath,replace):
    
    # 1) check if folder exist
    foldname = 'session_'+str(session_id)
    session = os.listdir(targetpath)
    if not foldname in os.listdir(targetpath):
        os.mkdir(os.path.join(targetpath,foldname))       
    
    # 2) check if the file exist
    filename = 'probe_'+ str(probe_id) +'_lfp.nwb'
    if not filename in os.listdir(os.path.join(targetpath,foldname)):
        shutil.copyfile(os.path.join(sourcepath,filename),os.path.join(targetpath,foldname,filename))
    else:
        if replace:
             shutil.copyfile(os.path.join(sourcepath,filename),os.path.join(targetpath,foldname,filename))   
    return True